# EMLOpt
Master Thesis in Artificial Intelligence at University of Bologna, a.y. 2021/2022

Daniele Verì, Michele Lombardi, Andrea Borghesi, Stefano Teso


# 📥 Dependencies

In [ ]:
#@title CPLEX

!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/69b1b89a73a7d0931fbfdb355eb147c3 -O cplex_studio1210.linux-x86-64.bin
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/97133b747b0114a4e3dba77ab26d68d5 -O response.properties

!pip install docplex
!sh cplex_studio1210.linux-x86-64.bin -f response.properties
!python3 /opt/ibm/ILOG/CPLEX_Studio1210/python/setup.py install

In [ ]:
#@title Tensorflow addons

!pip install tensorflow-addons

In [ ]:
#@title EMLlib

!git clone https://github.com/DanieleVeri/emllib.git

In [ ]:
#@title scikit-optimize
!pip install scikit-optimize

# 📑 Definitions

In [ ]:
#@title Base import and seed

import os
import math
import random
import numpy as np
import tensorflow as tf

import sys
if not 'emllib' in sys.path: sys.path.insert(1, 'emllib')

import pickle

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

##Problem

In [ ]:
#@title Problem class - quadratic + random slack

import docplex.mp.model as cpx

from skopt.sampler import Lhs
from skopt.space import Space

class ProblemQRS:

    def __init__(self, name, fun, input_type, input_bounds, constraint_cb=None, stocasthic=False):
        self.name = name
        self.fun = fun
        self.input_type = input_type
        self.input_bounds = input_bounds
        self.input_shape = len(self.input_bounds)
        self.constraint_cb = constraint_cb
        self.stocasthic = stocasthic

    def get_dataset(self, n_points):
        if self.constraint_cb is None:
            x = np.random.rand(n_points, self.input_shape)
            for i, b in enumerate(self.input_bounds):
                lb = b[0]
                ub = b[1]
                if self.input_type[i] == "int":
                    x[:,i] = np.random.randint(lb, high=ub, size=n_points)
                else:
                    x[:,i] *= ub - lb
                    x[:,i] += lb
            y = np.zeros((n_points))
            for i in range(n_points):
                y[i] = self.fun(x[i, :])
            return x, y

        # constrained dataset:
        x = np.zeros((n_points, self.input_shape))
        bounds = []
        for i, b in enumerate(self.input_bounds):
            if self.input_type[i] == "int":
                bounds.append((int(b[0]), int(b[1])))
            else:
                bounds.append((float(b[0]), float(b[1])))
                
        space = Space(bounds)
        lhs = Lhs(lhs_type="classic", criterion=None, iterations=1000)
        lhs_samples = lhs.generate(space.dimensions, n_points)

        p = 0
        while True:
            cplex = cpx.Model()
            xvars = []
            for i,b in enumerate(self.input_bounds):
                if self.input_type[i] == "int":
                    xvars.append(cplex.integer_var(lb=b[0], ub=b[1], name="x"+str(i)))
                else:
                    xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))
            
            csts = self.constraint_cb(cplex, xvars)
            # add slack
            avg_range = np.mean(np.diff(np.array(self.input_bounds), axis=1))
            for pc in csts:
                if pc[0].sense.value == 1: # <=
                    if pc[0].right_expr.is_constant() and not pc[0].right_expr.is_zero():
                        pc[0].right_expr -= np.random.uniform()*pc[0].right_expr
                    else:
                        pc[0].right_expr -= np.random.uniform()*avg_range
                elif pc[0].sense.value == 3: # >=
                    if pc[0].right_expr.is_constant() and not pc[0].right_expr.is_zero():
                        pc[0].right_expr += np.random.uniform()*pc[0].right_expr
                    else:
                        pc[0].right_expr += np.random.uniform()*avg_range
                print("\n", pc[0])
                cplex.add_constraint(*pc)

            ## quadratic random objective (boundaries)
            obj = 0
            for i, var in enumerate(xvars):
                obj += (var-lhs_samples[p][i]) ** 2
            cplex.set_objective("min", obj)
            
            # solve
            cplex.set_time_limit(30)
            sol = cplex.solve()
            if sol is None:
                print("infeasible")
                continue
            for i in range(self.input_shape):
                x[p, i] = sol["x"+str(i)]

            p += 1
            if p == n_points: break
        print("Initial points:\n", x)

        # eval fun
        y = np.zeros((n_points))
        for i in range(n_points):
            y[i] = self.fun(x[i, :])

        return x,y

    def get_grid(self, n_points):
        x_list = []
        for i, b in enumerate(self.input_bounds):
            lb = b[0]
            ub = b[1]
            if self.input_type[i] == "int":
                x_list.append(np.arange(lb, ub, max(1, (ub-lb)//n_points)))
            else:
                x_list.append(np.arange(lb, ub, (ub-lb)/n_points))
        x = np.array(np.meshgrid(*x_list)).reshape(self.input_shape,-1).T
        y = np.zeros((x.shape[0]))
        for i in range(x.shape[0]):
            y[i] = self.fun(x[i, :])
        return x, y

In [ ]:
#@title Problem class - convex interpolation

import docplex.mp.model as cpx
import pandas as pd

class Problem:

    def __init__(self, name, fun, input_type, input_bounds, constraint_cb=None, stocasthic=False):
        self.name = name
        self.fun = fun
        self.input_type = input_type
        self.input_bounds = input_bounds
        self.input_shape = len(self.input_bounds)
        self.constraint_cb = constraint_cb
        self.stocasthic = stocasthic

    def get_dataset(self, n_points):
        if self.constraint_cb is None:
            x = np.random.rand(n_points, self.input_shape)
            for i, b in enumerate(self.input_bounds):
                lb = b[0]
                ub = b[1]
                if self.input_type[i] == "int":
                    x[:,i] = np.random.randint(lb, high=ub, size=n_points)
                else:
                    x[:,i] *= ub - lb
                    x[:,i] += lb
            y = np.zeros((n_points))
            for i in range(n_points):
                y[i] = self.fun(x[i, :])
            return x, y

        # Constrained dataset:
        x = np.zeros((n_points, self.input_shape))
        bounds = []
        for i, b in enumerate(self.input_bounds):
            if self.input_type[i] == "int":
                bounds.append((int(b[0]), int(b[1])))
            else:
                bounds.append((float(b[0]), float(b[1])))

        n_points_boundaries = n_points // 2
        for p in range(n_points_boundaries):
            cplex = cpx.Model()
            xvars = []
            for i,b in enumerate(self.input_bounds):
                if self.input_type[i] == "int":
                    xvars.append(cplex.integer_var(lb=b[0], ub=b[1], name="x"+str(i)))
                else:
                    xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))
            
            csts = self.constraint_cb(cplex, xvars)
            for pc in csts:
                cplex.add_constraint(*pc)

            # linear random objective (boundaries)
            obj = 0
            for var in xvars:
                obj += var * (np.random.uniform()*2-1)
            cplex.set_objective("max", obj)
            
            # solve
            cplex.set_time_limit(30)
            sol = cplex.solve()
            for i in range(self.input_shape):
                x[p, i] = sol["x"+str(i)]

        # Interpolation  
        for p in range(n_points_boundaries, n_points):
            pidx = np.random.choice(n_points_boundaries, 2, replace=False)
            p0, p1 = x[pidx[0]], x[pidx[1]]
            step = np.random.uniform()
            df = pd.DataFrame([p0, [np.nan]*self.input_shape, p1], 
                              index=[0, step, 1])
            df = df.interpolate(method="index")
            x[p] = df.iloc[1].values

        # eval fun
        y = np.zeros((n_points))
        for i in range(n_points):
            y[i] = self.fun(x[i, :])

        return x,y

    def get_grid(self, n_points):
        x_list = []
        for i, b in enumerate(self.input_bounds):
            lb = b[0]
            ub = b[1]
            if self.input_type[i] == "int":
                x_list.append(np.arange(lb, ub, max(1, (ub-lb)//n_points)))
            else:
                x_list.append(np.arange(lb, ub, (ub-lb)/n_points))
        x = np.array(np.meshgrid(*x_list)).reshape(self.input_shape,-1).T
        y = np.zeros((x.shape[0]))
        for i in range(x.shape[0]):
            y[i] = self.fun(x[i, :])
        return x, y

##TFP utils

In [ ]:
#@title Build and plot

from matplotlib import cm
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

def build_probabilistic_regressor(input_shape, depth=4, width=20):
    mdl = tf.keras.Sequential()
    mdl.add(tf.keras.layers.Input(shape=(input_shape,), dtype='float32'))
    for i in range(depth):
        mdl.add(tf.keras.layers.Dense(width, activation='relu'))
    mdl.add(tf.keras.layers.Dense(2, activation='linear'))
    lf = lambda t: tfp.distributions.Normal(loc=t[:, :1], 
                                            scale=tf.keras.backend.exp(t[:, 1:]))
    mdl.add(tfp.layers.DistributionLambda(lf))
    return mdl

def dlambda_likelihood(y_true, dist):
    return -dist.log_prob(y_true)

def plot_prob_predictions(mdl, x, y):
    prob_pred = mdl(np.expand_dims(x, axis=1))
    pred = prob_pred.mean().numpy().ravel()
    std_pred = prob_pred.stddev().numpy().ravel()
    plt.plot(x, y, c="grey")
    plt.plot(x, pred)
    plt.fill_between(x, pred-std_pred, pred+std_pred, 
                    alpha=0.3, color='tab:blue', label='+/- std')

## EML utils

In [ ]:
#@title Parse TFP model

from eml.net.reader import keras_reader

def parse_tfp(model):
    in_shape = model.input_shape[1]
    mdl_no_dist = tf.keras.Sequential()
    mdl_no_dist.add(tf.keras.layers.Input(shape=(in_shape,), dtype='float32'))
    for i in range(len(model.layers)-2):
        w = model.layers[i].weights[1].shape[0]
        mdl_no_dist.add(tf.keras.layers.Dense(w, activation='relu'))
    mdl_no_dist.add(tf.keras.layers.Dense(2, activation='linear'))

    mdl_no_dist.set_weights(model.get_weights())

    nn = keras_reader.read_keras_sequential(mdl_no_dist)
    return nn

In [ ]:
#@title Bounds

from eml.net.process import ibr_bounds, fwd_bound_tighthening

def propagate_bound(bkd, parsed_model, bounds):
    bounds = np.array(bounds)
    parsed_model.layer(0).update_lb(bounds[:,0])
    parsed_model.layer(0).update_ub(bounds[:,1])
    fwd_bound_tighthening(bkd, parsed_model, timelimit=30)
    print("Model output bounds:\n",parsed_model)
    return parsed_model

In [ ]:
#@title Encode model

from eml.backend import cplex_backend
from eml.net import embed

def embed_model(bkd, cplex, parsed_model, vtype, bounds):
    mean_lb = parsed_model.layer(-1).lb()[0]  # bounds computed with propagate bounds method
    mean_ub = parsed_model.layer(-1).ub()[0]
    std_lb = parsed_model.layer(-1).lb()[1]
    std_ub = parsed_model.layer(-1).ub()[1]

    xvars = []
    for i,b in enumerate(bounds):
        xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))

    yvars = [cplex.continuous_var(lb=mean_lb, ub=mean_ub, name="out_mean"), 
            cplex.continuous_var(lb=std_lb, ub=std_ub, name="out_std")]

    embed.encode(bkd, parsed_model, cplex, xvars, yvars, 'nn')
    return xvars, yvars

In [ ]:
#@title PWL helper

from eml.util import encode_pwl
from scipy.stats import norm


def pwl_exp(bkd, cplex, var, nnodes=7):
    xx = np.linspace(var.lb, var.ub, nnodes)
    yy = np.array(list(map(math.exp, xx)))
    v = [cplex.continuous_var(lb=0, ub=np.max(yy), name="exp_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_abs(bkd, cplex, var):
    xx = np.array([-1, 0, 1])
    yy = np.array([1, 0, 1])
    v = [cplex.continuous_var(lb=0, ub=1), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_normal_cdf(bkd, cplex, var, nnodes=11):
    xx = np.linspace(var.lb, var.ub, nnodes)
    yy = np.array(list(map(norm.cdf, xx)))
    v = [cplex.continuous_var(lb=0, ub=1, name="ncdf_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_normal_pdf(bkd, cplex, var, nnodes=11):
    xx = np.linspace(var.lb, var.ub, nnodes)
    yy = np.array(list(map(norm.pdf, xx)))
    v = [cplex.continuous_var(lb=0, ub=1, name="npdf_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_sample_dist(bkd, cplex, vars, samples, vtype, bounds, nnodes=20):
    mapf = lambda x: np.min(np.sum(np.abs(samples - x), axis=1)) / len(bounds)
    p = Problem(None, mapf, vtype, bounds)
    x,y = p.get_grid(nnodes)
    ub = np.diff(np.array(bounds), axis=1)
    ub = np.sum(ub)
    v = [cplex.continuous_var(lb=0, ub=ub, name="dist_out")]+vars
    encode_pwl(bkd, cplex, v, [y,*x.T])
    return v[0]

## Optimization loop

In [ ]:
#@title Base Experiment

import time
import tensorflow_addons as tfa

class BaseExperiment:
    def __init__(self, problem, starting_points, iterations, 
                 epochs, lr, weight_decay, depth, width, batch_size, 
                 solver_timeout):
        set_seed()
        self.problem = problem
        self.starting_points = starting_points
        self.iterations = iterations
        self.epochs = epochs
        self.lr = lr
        self.weight_decay = weight_decay
        self.depth = depth
        self.width = width
        self.batch_size = batch_size
        self.solver_timeout = solver_timeout
        self.x_samples, self.y_samples = problem.get_dataset(starting_points)

    def train(self, keras_mdl):
        raise NotImplementedError

    def solver_optimization(self, keras_mdl):
        raise NotImplementedError

    def solution_log(self, solution):
        raise NotImplementedError

    def normalize_X(self, x):
        bounds = np.array(self.problem.input_bounds)
        return (x-bounds[:,0]) / (bounds[:,1]-bounds[:,0])

    def normalize_Y(self, y):
        min, max = np.min(self.y_samples), np.max(self.y_samples)
        return (y-min) / (max-min)

    def reverse_normalize_X(self, norm_x, milp_expr=False):
        bounds = np.array(self.problem.input_bounds)
        if milp_expr:  
            # for cplex variables, np array not supported
            bounds_range = np.squeeze(np.diff(bounds, axis=1))
            xvars = []
            for i,x in enumerate(norm_x):
                xvars.append(x * bounds_range[i] + bounds[i,0])
            return xvars
        return (bounds[:,1]-bounds[:,0])*norm_x + bounds[:,0]

    def reverse_normalize_Y(self, norm_y, stddev=False):
        min, max = np.min(self.y_samples), np.max(self.y_samples)
        if stddev:
            return norm_y*(max-min)
        return (max-min)*norm_y + min

    def plot(self, hstory, keras_mdl):
        if self.problem.input_shape <= 2:
            x,y = self.problem.get_grid(100)
            prob_pred = keras_mdl(self.normalize_X(x))
            pred = prob_pred.mean().numpy().ravel()
            pred = self.reverse_normalize_Y(pred)
            std_pred = prob_pred.stddev().numpy().ravel()
            std_pred = self.reverse_normalize_Y(std_pred, stddev=True)

        plt.plot(hstory.history["loss"])
        plt.savefig('train_loss.png')
        plt.show()
        
        fig = plt.figure(figsize=(15,10))
        if self.problem.input_shape == 1:   # 1D domain
            plt.xlim(self.problem.input_bounds[0])
            x = np.squeeze(x)
            plt.plot(x, y, c="grey")
            plt.plot(x, pred)
            plt.fill_between(x, pred-std_pred, pred+std_pred, 
                            alpha=0.3, color='tab:blue', label='+/- std')
            plt.scatter(self.x_samples, self.y_samples, c="orange")
            plt.legend(["GT", "predicted mean", "predicted CI", "samples"])
            plt.savefig('chart.png')
            plt.show()
        elif self.problem.input_shape == 2:   # 2D domain
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(self.x_samples[:,0], self.x_samples[:,1], self.y_samples, color="orange")
            ax.scatter(x[:,0], x[:,1], y, alpha=0.15, color="lightgrey")
            ax.scatter(x[:,0], x[:,1], pred, alpha=0.3)
            ax.scatter(x[:,0], x[:,1], pred-std_pred, alpha=0.3, color="lightblue")
            ax.scatter(x[:,0], x[:,1], pred+std_pred, alpha=0.3, color="lightblue")
            ax.view_init(elev=15, azim=60)
            plt.legend(["samples", "GT", "predicted mean", "predicted CI"])
            plt.savefig('chart.png')
            plt.show()
        else:
            print("Plot not available for high dimensional domains.")
            print(f"X:\n{self.x_samples}\nY:\n{self.y_samples}")
            prob_pred = keras_mdl(self.normalize_X(self.x_samples))
            mu = self.reverse_normalize_Y(prob_pred.mean().numpy().ravel())
            print("mu pred\n",mu)
            print("diff\n",self.y_samples-mu)

    def run(self):
        not_improve = 0
        for iteration in range(self.iterations):
            # build NN
            print(f"Iteration {iteration}:", "="*20)
            optimizer = tfa.optimizers.AdamW(weight_decay=self.weight_decay,
                                             learning_rate=self.lr)
            keras_mdl = build_probabilistic_regressor(
                self.problem.input_shape, self.depth, self.width)
            keras_mdl.compile(optimizer=optimizer, loss=dlambda_likelihood)
            keras_mdl.summary()

            # train
            train_time = time.time()
            keras_mdl, hstory = self.train(keras_mdl)
            train_time = time.time() - train_time

            # plot loss and predictions if domain dim <= 2
            self.plot(hstory, keras_mdl)

            # MILP solver
            solver_time = time.time()
            sol = self.solver_optimization(iteration, not_improve, keras_mdl)
            solver_time = time.time() - solver_time

            if sol is None:
                print(f'Not feasible')
                break
            sol.solve_details.print_information()

            # retrieve solution
            opt_x = np.zeros(self.problem.input_shape)
            for i in range(self.problem.input_shape):
                opt_x[i] = sol["x"+str(i)]
            opt_x = self.reverse_normalize_X(opt_x)
            
            # retrieve integer solutions
            for i in range(self.problem.input_shape):
                if self.problem.input_type[i] == "int":
                    opt_x[i] = sol["int_x"+str(i)]

            # check if repeated data point and query obj function
            dists = np.sum(np.abs(self.x_samples - np.expand_dims(opt_x, 0)), axis=1)
            dmin = np.min(dists)
            dargmin = np.argmin(dists)
            # For non stocasthic problems, avoid resampling the same points, but giving much sample weight
            if not self.problem.stocasthic and dmin <= 1e-6: 
                print("Preventing query for an already known point !")
                opt_y = self.y_samples[dargmin]
            else:
                # BBF query
                opt_y = self.problem.fun(opt_x)

            # log new point
            emean = self.reverse_normalize_Y(sol['out_mean'])
            print(f"opt input={opt_x}, expected mean={emean}, gt={opt_y}")
            print(f"train time: {train_time}, solver time: {solver_time}")

            # not improve counter
            if opt_y < np.min(self.y_samples):
                not_improve = 0
            else:
                not_improve += 1
            
            # add new point to the dataset
            self.x_samples = np.concatenate((self.x_samples, np.expand_dims(opt_x, 0)))
            self.y_samples = np.append(self.y_samples, opt_y)

            # log and WandB
            if iteration == 0 and ENABLE_WANDB: # log starting points before the first iteration
                for j in range(self.starting_points):
                    wandb.log({"x_sample": self.x_samples[j], "y_sample": self.y_samples[j]})
                
            self.solution_log(sol)

            if ENABLE_WANDB: 
                if self.problem.input_shape <= 2:
                    wandb.log({"train_predictions": wandb.Image('chart.png')}, commit=False)
                wandb.log({
                    "train_loss": wandb.Image('train_loss.png'),
                    "x_sample": opt_x, "y_sample": opt_y, "y_min": np.min(self.y_samples),
                    "train_time": train_time, "solver_time": solver_time
                })
        # == end opt loop
        print(f"Min found: {np.min(self.y_samples)} in {np.argmin(self.y_samples)+1-self.starting_points} iterations")
        if ENABLE_WANDB:
            wandb.log({
                "min_found": np.min(self.y_samples),
                "n_iterations": np.argmin(self.y_samples)+1-self.starting_points
            })

        # Dump points for future studies
        dump_points = list(zip(self.x_samples, self.y_samples))
        with open('points.pkl', 'wb') as f:
            pickle.dump(dump_points, f)
        if ENABLE_WANDB:
            artifact = wandb.Artifact('datapoints', type='points')
            artifact.add_file("points.pkl")
            wandb.log_artifact(artifact)

        ymin = np.min(self.y_samples)
        yargmin = np.argmin(self.y_samples)
        xmin = self.x_samples[yargmin]
        return ymin, xmin

In [ ]:
#@title Base UCB

class BaseUCBExperiment(BaseExperiment):

    def __init__(self, beta_ucb, *args, **kwargs):
        super(BaseUCBExperiment, self).__init__(*args, **kwargs)
        self.beta = beta_ucb

    def solver_optimization(self, iteration, not_improve, keras_mdl):
        print("UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(cplex, rev_norm_xvars)
        
        ucb = -yvars[0] + self.beta * stddev
        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()

        print(cplex.solve_details)
        cplex.print_information()
        return sol

    def solution_log(self, solution):
        print(f"UCB: {solution.objective_value}")
        if ENABLE_WANDB: 
            wandb.log({
                "ucb": solution['ucb'],
                "exp_err": solution['exp_out'] - math.exp(solution['out_std'])
            }, commit=False)

In [ ]:
#@title Stop CI

class StopCICB(tf.keras.callbacks.Callback):

    def __init__(self, threshold, *args, **kwargs):
        super(StopCICB, self).__init__(*args, **kwargs)
        self.threshold = threshold
        self.x = None

    def on_epoch_end(self, epoch, logs={}):
        prob_pred = self.model(self.x)
        std_pred = prob_pred.stddev().numpy().ravel()
        if epoch % 100 == 0:
            print(epoch, "mean stddev", np.mean(std_pred))
        if np.mean(std_pred) < self.threshold:
            print("break condition on epoch", epoch, "with mean stddev", np.mean(std_pred))
            self.model.stop_training = True

class StopCI(BaseUCBExperiment):

    def __init__(self, ci_threshold, *args, **kwargs):
        super(StopCI, self).__init__(*args, **kwargs)
        self.ci_threshold = ci_threshold
        stop_ci = StopCICB(self.ci_threshold)
        self.cb = [stop_ci]

    def train(self, keras_mdl):
        print("Stop CI training...")
        bs = self.batch_size if self.batch_size else self.x_samples.shape[0]

        norm_x = self.normalize_X(self.x_samples)
        norm_y = self.normalize_Y(self.y_samples)

        self.cb[0].x = norm_x

        hstory = keras_mdl.fit(norm_x, norm_y,
                batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb)

        return keras_mdl, hstory

In [ ]:
#@title Beta decay

class BetaDecay(StopCI):

    def __init__(self, lns_fixed, *args, **kwargs):
        super(BetaDecay, self).__init__(*args, **kwargs)
        self.lns_fixed = lns_fixed

    def solver_optimization(self, iteration, not_improve, keras_mdl):
        print("Distance based UCB solver...")

        ## K-Lipschitz with L1 dist
        k_lip = 0
        qlist = []
        for i in range(self.x_samples.shape[0]):
            for j in range(i+1, self.x_samples.shape[0]):
                delta_y = np.abs(self.y_samples[i]-self.y_samples[j])
                delta_x = np.sum(np.abs(self.x_samples[i]-self.x_samples[j]))/self.problem.input_shape
                if delta_x >= 1e-6: #eps to avoid inf
                    k_lip = max(k_lip, delta_y/delta_x)
                    qlist.append(delta_y/delta_x)
        print("K-Lipschitz max:", k_lip, 
              "95 percentile:", np.percentile(qlist, 95, interpolation='linear'))
        
        solution = None
        feature_fixed = self.problem.input_shape - (self.problem.input_shape // self.lns_fixed)
        for lns_i in range(self.lns_fixed):
            print("@"*20, "LSN iteration:", lns_i)
            cplex = cpx.Model()
            bkd = cplex_backend.CplexBackend()
            norm_x = self.normalize_X(self.x_samples)

            ## Beta
            if self.beta is not None:
                self.current_beta = self.beta
            else:
                self.current_beta = k_lip * ((1-iteration/ITERATIONS)**4)

            ## LNS
            best = norm_x[np.argmin(self.y_samples)]
            selection = np.random.choice(self.problem.input_shape, feature_fixed, replace=False)
            print("LNS Random selection", selection, "best:", best)
            pbounds = [[0,1]]*self.problem.input_shape
            for v in selection:
                pbounds[v] = [best[v], best[v]]
            
            ## NN encoding
            parsed_mdl = parse_tfp(keras_mdl)
            print("Propagating bounds...")
            propagate_bound(bkd, parsed_mdl, pbounds)
            xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                    self.problem.input_type,
                                    pbounds)

            # Handle integer variables
            for idx, bound in enumerate(self.problem.input_bounds):
                if self.problem.input_type[idx] == "int":
                    int_cst = cplex.integer_var(lb=bound[0], ub=bound[1], name="int_x"+str(idx))
                    cplex.add_constraint(int_cst == xvars[idx]*(bound[1]-bound[0])+bound[0])
            
            ## EXP PWL
            stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

            #################### Distance section
            # Sample distance
            sample_distance_list = []
            for row in range(norm_x.shape[0]):
                current_sample_dist = 0
                for feature in range(norm_x.shape[1]):
                    current_sample_dist += cplex.abs(norm_x[row, feature] - xvars[feature])
                sample_distance_list.append(current_sample_dist)
            
            ## triangular inequality
            def l1_dist(a,b): 
                return np.sum(np.abs(a-b))
            for id_xy in range(len(sample_distance_list)):
                s_xy = sample_distance_list[id_xy]
                for id_xz in range(id_xy, len(sample_distance_list)):
                    s_xz = sample_distance_list[id_xz]
                    cplex.add_constraint(s_xy <= s_xz + l1_dist(norm_x[id_xy],norm_x[id_xz]), "traingular inequality")

            # Min distance
            min_dist = cplex.continuous_var(lb=0, ub=self.problem.input_shape, name="dist")
            for current_sample_dist in sample_distance_list:
                cplex.add_constraint(current_sample_dist >= min_dist, "min dist")  # scale down the l1 dist with dimensions
            #################### END Distance section
            
            # Problem contraints
            if self.problem.constraint_cb is not None:
                rev_norm_xvars = self.reverse_normalize_X(xvars, True)
                csts = self.problem.constraint_cb(cplex, rev_norm_xvars)
                for pc in csts:
                    cplex.add_constraint(*pc)

            # UCB
            ucb = -yvars[0] + \
                self.current_beta * stddev + \
                (self.current_beta/self.problem.input_shape) * min_dist

            cplex.set_objective('max', ucb)
            cplex.set_time_limit(self.solver_timeout)

            cplex.parameters.mip.strategy.nodeselect = 2

            # CPLEX log
            cplex.context.solver.verbose = 5
            cplex.context.solver.log_output = True

            cplex.print_information()
            box = sys.stdout
            sys.stdout = open('cplex_model.txt', 'w')
            cplex.prettyprint()
            sys.stdout.close()
            sys.stdout = box

            print("Solving...")
            sol = cplex.solve()

            if sol is not None:
                if solution is not None:
                    if sol.objective_value > solution.objective_value:
                        solution = sol
                else:
                    solution = sol

        return solution

    def solution_log(self, solution):
        print("Beta:", self.current_beta)
        print("Normalized Dist:", solution['dist'])
        print(f"UCB: {solution.objective_value}")
        print("mean:", solution['out_mean'])
        print("logstd:", solution['out_std'])
        print("truestd:", math.exp(solution['out_std']))
        print("stddev:", solution['exp_out'])
        if ENABLE_WANDB: 
            wandb.log({
                "ucb": solution.objective_value,
                "norm_dist": solution['dist'],
                "mean": solution['out_mean'],
                "stddev": solution['exp_out'],
                "exp_err": solution['exp_out'] - math.exp(solution['out_std']),
                "beta_ucb": self.current_beta
            }, commit=False)

# 🚀 Experiments

In [ ]:
#@title 📈 Parameters and WandB

ITERATIONS =                                  50#@param {type:"integer"}
STARTING_POINTS =                              5#@param {type:"integer"}

EPOCHS =                                    999#@param {type:"integer"}  

LR =                                        1e-3#@param {type:"number"}
WEIGHT_DECAY =                                        1e-4#@param {type:"number"}
BATCH_SIZE = None                               #@param {type:"raw"} None = all samples in 1 batch
DEPTH =                                        1#@param {type:"integer"}
WIDTH =                                       200#@param {type:"integer"}

BETA_UCB =                                     1#@param {type:"raw"}

SOLVER_TIMEOUT =                              120#@param {type:"integer"}

CI_THRESHOLD =              0.05#@param {type:"raw"} 
LNS_FIXED =              1#@param {type:"integer"} 
#@markdown ---
# set if you plan to log on wandb
ENABLE_WANDB = True                            #@param {type:"boolean"}        
EXPERIMENT_NAME = "example_experiment"          #@param {type:"string"}

if ENABLE_WANDB and "wandb" not in sys.modules:
    !pip install wandb > /dev/null
    !wandb login
    import wandb

def init_wandb(experiment_name, run_id):
    if run_id is not None: 
        wandb.init(project='eml', id=run_id, resume='allow')
    else:
        wandb.init(project='eml', name=experiment_name)

        wandb.config.iterations = ITERATIONS
        wandb.config.starting_points = STARTING_POINTS
        wandb.config.epochs = EPOCHS
        wandb.config.lr = LR
        wandb.config.weight_decay = WEIGHT_DECAY
        wandb.config.batch_size = BATCH_SIZE
        wandb.config.depth = DEPTH
        wandb.config.width = WIDTH
        wandb.config.beta_ucb = BETA_UCB
        wandb.config.solver_timeout = SOLVER_TIMEOUT
        wandb.config.ci_threshold = CI_THRESHOLD
        wandb.config.lns_fixed = LNS_FIXED

def run_experiment(target):
    instance = BetaDecay(
        LNS_FIXED,
        CI_THRESHOLD, 
        BETA_UCB, 
        target, STARTING_POINTS, ITERATIONS, 
        EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
        SOLVER_TIMEOUT)
    return instance.run()

if ENABLE_WANDB:
    init_wandb(EXPERIMENT_NAME, None)


In [ ]:
#@title ▶️ Run experiment

# objective function
def obj_ackley(x):
    x1,x2 = x[0],x[1]
    return -20*math.exp(-0.2*math.sqrt(0.5*(x1*x1+x2*x2))) - \
        math.exp(0.5*(math.cos(2*math.pi*x1)+math.cos(2*math.pi*x2))) + \
        math.e + 20

# constraint
def cst_disk(cplex, xvars):     # x1^2 + x2^2 < r^2
    x1,x2 = xvars
    r = 2
    return [[x1*x1 + x2*x2 <= r*r, "center_dist"]]

# define problem
example_problem = Problem(
        "example_problem", 
        obj_ackley, 
        ["real", "real"],
        [[-3, 3], [-2, 2]],
        cst_disk, 
        stocasthic=False)

#---------------- Start optimization ----------------#
run_experiment(example_problem)
#----------------------------------------------------#